# Ensamblaje y Random Forest

Imagina que haces una pregunta compleja a miles de personas aleatorias y, después, ponderas sus respuestas. En muchos casos, te encontrarás con que esta respuesta ponderada es mejor que las respuestas de un experto. Esto se denomina sabiduría colectiva.

De modo similar, si ponderas las predicciones mejoras que con el mejor predictor individual. Un grupo de predictores se denomina ensamble; así se conoce como técnica de ensamblaje.

Como ejemplo de método de ensamblaje, puedes entrenar un grupo de árboles de decisión, cada uno en un subconjunto aleatorio del conjunto de entrenamiento aleatorio. Para hacer predicciones, obtienes los predictores de todos los árboles individuales y, después, predices la clase que consiga más votos. Un ensamble de árboles se denomina _Random Forest_, y pese a su simplicidad, es uno de los algoritmos de _machine learning_ más potentes en la actualidad.

Los métodos de ensamblaje más populares son _bagging_, _boosting_ y _stacking_.

## Clasificadores de votación

Supongamos que has entrenados varios clasificadores y cada uno ha consegudo una exactitud al rededor del 80%. Puede que tengas una Regresión Logística, un clasificador SVM, un _Random Forest_, un kNN y algunos más. Una manera muy sencilla de crear un clasificador aún más potente es sumar las predicciones de clasificadores y predecir la clase que obtiene más votos. Este clasificador por votos de la mayoría se llama clasificador _hard voting_.

<img src="https://i.stack.imgur.com/W7UmY.png" width="500" />

De estra manera, este clasificador consigue a menudo una exactitud mayor que el mejor clasificador del ensamble.

Si tenemos una moneda ligeramente desequilibrada que tiene un 51% de posibilidades de caer cara. Si la lanzamos 1.000 veces, en general saldrían más o menos 510 caras y 490 cruces y, por lo tanto, una mayoría de caras. Si haces los cálculos, la posibilidad de tener mayoría de caras después de 1.000 lanzamientos se acerca al 75%, y cuando más se lance la moneda más alta es esta probabilidad (ej. con 10.000 lanzamientos, sube al 97%). Esto se debe a la Ley de los grandes número TLC.

In [45]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=1000, noise=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [5]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [9]:
voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

Exactitud de cada clasificador

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf) :
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9
RandomForestClassifier 0.9
SVC 0.9333333333333333
VotingClassifier 0.9333333333333333


Si todos los clasificadores son capaces de estimar las probabilidades de clase (es decir, todos tienen el método predict_proba()), se puede solicitar que prediga a través de la clase con más alta probabilidad, esto se denomina _soft voting_. A menudo consigue un rendimiento más alto que el _hard voting_ por que da más peso a los botos que son más seguros.

In [36]:
voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf)],
    voting = 'soft'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [39]:
for clf in (log_clf, rnd_clf, voting_clf) :
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9
RandomForestClassifier 0.9333333333333333
VotingClassifier 0.9333333333333333


## Bagging y Pasting

Una forma de obtener un conjunto diverso de clasificadores es usar algoritmos de entrenamientos muy diferentes, como ya hemos visto. Otro enfoque es utilizar el mismo algoritmo para todos los predictores y entrenarlos en subconjuntos aleatorios distintos al conjunto de entrenamiento. Cuando se realiza un muestreo con reemplazo, este método se denomina __bagging__ (boostrap aggregation). Cuando el muestreo se realiza sin reemplazo se denomina __pasting__.

Dicho de otro modo, tanto como _bagging_ como _pasting_ permiten entrenar instancias para que se muestreen varias veces a través de multiples predictores, pero solo el _bagging_ permite entrenar instancias para que se muestreen varias veces para el mismo predictor.

<img src="https://miro.medium.com/max/1400/1*iskng0M2Qv9GF0CADcl0Ww.png" width="600" />

Una vez todos los predictores están entrenados, el ensamble puede hacer una predicción para una nueva instancia con solo agregar las predicciones de todos los predictores. La función de agregación suele ser el modo estadístico (el más frecuente además de _hard voting_) para clasificación o el promedio para la regresión. Cada predictor individual tiene un sesgo más alto que si se entrenase con el conjunto de datos indivudual, pero la agregación reduce el sesgo como la varianza.

### Bagging y pasting en Sckit-Learn

Se desarrolla un ensamblador de 500 clasificadores de arboles de decisión, Cada uno se entrena en 100 instancias de entrenamiento, aleatoriamente muestradas del conjunto de entrenamiento con reemplazo (_bagging_).

In [47]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

Nota: El BaggingClassifier realiza _soft voting_ de forma automática, si el clasificador lo permite.

Tal como se observa en la imagen, un ensamble con _bagging_ de 500 arboles es capaz de generalizar mucho mejor que un solo arbol de decisión, tiene un mayor sesgo pero una menor varianza, osea, comete aproximadamente el mismo número de errores en el conjunto de entrenamiento, pero el límite de decisión es menos irregular.

<img src="https://miro.medium.com/max/1006/1*dCU5rsihNgN6rZrR3jpPwA.png" width="500" />

El método _bagging_ acaba con un sesgo ligeramente más alto que el _pasting_; pero la diversidad extra también significa que los predictores acaban estando menos correlacionados, así la varianza del ensamble se reduce. En general, _bagging_ tiene resultadores mejores, por lo que suele preferirse.

Sin embargo, con el tiempo y CPU suficiente, se puede utilizar validación cruzada para evaluar tanto _bagging_ como _pasting_ para seleccionar el que mejor funcione.

### Evaluación fuera de la bolsa

Con _bagging_, algunas instancais pueden muestrarse varias veces para cualquier predictor dado, mientras que otros pueden no muestrarse en absoluto. Por defecto, un BaggingClassifier muestrea m instancias de entrenamiento con reemplazo (boostrap=True), donde em es el tamaño del conjunto de entrenamiento. Eso significa que solo al rededor del 63% de las instancias de entrenamiento se muestran para cada predictor. Las no muestreadas se conocen como "fuera de la bolsa" (out-of-bag, oob).

## Random Forest

El _Random Forest_ es un ensamble de árboles de decisión, entrenado, por lo general, mediante el método _bagging_. Habitualmente con max_samples establecidos como el tamaño del conjunto de entrenamiento. La función DecisionTreeClassifier es más conveniente y esta más optimizado para árboles.

In [48]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rd = rnd_clf.predict(X_test)

El algoritmo introduce una aleatoriedad extra cuando hace crecer los árboles; en vez de buscar la mejor característica cuando divide un nodo,  busca la mejor característica de un subconjunto de características. Teniendo un resultado con mas diversidad de arboles, lo que compensa un sesgo más alto por una varianza más baja.